In [1]:
# You'll need the requests library in order to fully utilize bs4.
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
import time
import random


In [119]:
# Target web page:
url = "https://www.imdb.com/title/"
# Establishing the connection to the web page:
response = requests.get(url)
# response.text[:50]
soup = BeautifulSoup(response.text, 'lxml')

imdb_scrape = pd.DataFrame(columns=['tconst','newurl','rating','votes','year','mpaarating','runtime','rlsdt','budget','worldwide','soup_text'])
imdb_errors = pd.DataFrame(columns=['tconst'])

In [2]:
Disney_IMDB = pd.read_csv('../Bens_Data/Updated_IMDB_Disney_442.csv')

In [3]:
Disney_IMDB.drop(columns=['Unnamed: 0'], inplace=True)

In [4]:
Disney_IMDB.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   orig_index      442 non-null    int64  
 1   DFL_title       442 non-null    object 
 2   tconst          442 non-null    object 
 3   titleType       442 non-null    object 
 4   primaryTitle    442 non-null    object 
 5   originalTitle   442 non-null    object 
 6   startYear       442 non-null    int64  
 7   runtimeMinutes  442 non-null    int64  
 8   genres          442 non-null    object 
 9   lower_title     442 non-null    object 
 10  averageRating   442 non-null    float64
 11  numVotes        442 non-null    float64
 12  _merge          442 non-null    object 
dtypes: float64(2), int64(3), object(8)
memory usage: 45.0+ KB


In [5]:
Disney_IMDB.head(5)

,orig_index,DFL_title,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,lower_title,averageRating,numVotes,_merge
0,0,101 Dalmatians,tt0115433,movie,101 Dalmatians,101 Dalmatians,1996,103,"Adventure,Comedy,Crime",101 dalmatians,5.7,109712.0,both
1,5,102 Dalmatians,tt0211181,movie,102 Dalmatians,102 Dalmatians,2000,100,"Adventure,Comedy,Family",102 dalmatians,4.8,37056.0,both
2,10,"20,000 Leagues Under the Sea",tt0046672,movie,"20,000 Leagues Under the Sea","20,000 Leagues Under the Sea",1954,127,"Adventure,Drama,Family",20000 leagues under the sea,7.2,33109.0,both
3,24,A Bug's Life,tt0120623,movie,A Bug's Life,A Bug's Life,1998,95,"Adventure,Animation,Comedy",a bugs life,7.2,284538.0,both
4,74,A Christmas Carol,tt1067106,movie,A Christmas Carol,A Christmas Carol,2009,96,"Adventure,Animation,Comedy",a christmas carol,6.8,112582.0,both


In [6]:
dups_df = Disney_IMDB[Disney_IMDB['tconst'].isin(Disney_IMDB['tconst'].value_counts()[Disney_IMDB['tconst'].value_counts()>2].index)]
print(dups_df.shape)
dups_df.head(30)

(0, 13)


,orig_index,DFL_title,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,lower_title,averageRating,numVotes,_merge


In [125]:
for const in Disney_IMDB['tconst']:
    
#     found1 = imdb_scrape[imdb_scrape['tconst'].str.contains(const)]
#     found2 = imdb_errors[imdb_errors['tconst'].str.contains(const)]
#     summ = found1.count().sum() + found2.count().sum()
    
#     if summ < 1:
    
        # const = "tt0115433"
        # const = "tt0029583"
        # const = "tt1067106"

        t= random.uniform(1, 5)
        time.sleep(t)
        newurl = url + const
        response = requests.get(newurl)
        soup = BeautifulSoup(response.text, 'lxml')

        rtspan = soup.find_all('span', {'class' : 'sc-7ab21ed2-1 jGRxWM'})
        rating = rtspan[0].get_text()

        vtdiv = soup.find_all('div', {'class' : 'sc-7ab21ed2-3 dPVcnq'})
        votes = vtdiv[0].text

        # mpaaspan = soup.find_all('span', {'class' : 'ipc-metadata-list-item__list-content-item'})
        # mpaarating = mpaaspan[2].get_text()
        try:

            metaul = soup.find_all('ul', {'data-testid' : 'hero-title-block__metadata'})
            year = metaul[0].find_all('li')[0].find_all('span')[0].get_text()
            mpaarating = metaul[0].find_all('li')[1].find_all('span')[0].get_text()
            runtime = metaul[0].find_all('li')[2].get_text()
            rlsdt = soup.find_all('li', {'data-testid' : 'title-details-releasedate'})[0].find_all('a')[1].get_text()
            budget = soup.find_all('li', {'data-testid' : 'title-boxoffice-budget'})[0].find_all('li')[0].get_text()
            worldwide = soup.find_all('li', {'data-testid' : 'title-boxoffice-cumulativeworldwidegross'})[0].find_all('li')[0].get_text()
            print(newurl)
            print(rating)
            print(votes) 
            print(year) 
            print(mpaarating) 
            print(runtime) 
            print(rlsdt) 
            print(budget) 
            print(worldwide) 
            imdb_scrape.loc[len(imdb_scrape.index)] = [const, newurl, rating, votes, year ,mpaarating ,runtime ,rlsdt ,budget ,worldwide, soup]

        except:
            imdb_errors.loc[len(imdb_errors.index)] = [const]
            print(f"ERROR ON: {const}")

    

https://www.imdb.com/title/tt0115433
5.7
110K
1996
G
1h 43m
November 27, 1996 (United States)
$75,000,000 (estimated)
$320,689,294
https://www.imdb.com/title/tt0211181
4.8
37K
2000
G
1h 40m
November 22, 2000 (United States)
$85,000,000 (estimated)
$183,611,771
ERROR ON: tt0046672
https://www.imdb.com/title/tt0120623
7.2
285K
1998
G
1h 35m
November 25, 1998 (United States)
$120,000,000 (estimated)
$363,258,859
https://www.imdb.com/title/tt1067106
6.8
113K
2009
PG
1h 36m
November 6, 2009 (United States)
$200,000,000 (estimated)
$325,286,646
ERROR ON: tt0106868
https://www.imdb.com/title/tt0113198
6.9
52K
1995
G
1h 18m
April 7, 1995 (United States)
$18,000,000 (estimated)
$35,348,597
https://www.imdb.com/title/tt0113538
4.7
6.2K
1995
PG
1h 29m
August 11, 1995 (United States)
$15,000,000 (estimated)
$13,406,717
ERROR ON: tt0058657
https://www.imdb.com/title/tt1620680
4.2
44K
2018
PG
1h 49m
March 9, 2018 (United States)
$100,000,000 (estimated)
$132,675,864
https://www.imdb.com/title/tt0092

In [126]:
# const = "tt1067106"
# found1 = imdb_scrape[imdb_scrape['tconst'].str.contains(const)]
# found2 = imdb_errors[imdb_errors['tconst'].str.contains(const)]
# summ = found1.count().sum() + found2.count().sum()
# print(summ)

In [61]:
imdb_scrape.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240 entries, 0 to 239
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tconst      240 non-null    object
 1   newurl      240 non-null    object
 2   rating      240 non-null    object
 3   votes       240 non-null    object
 4   year        240 non-null    object
 5   mpaarating  240 non-null    object
 6   runtime     240 non-null    object
 7   rlsdt       240 non-null    object
 8   budget      240 non-null    object
 9   worldwide   240 non-null    object
 10  soup_text   240 non-null    object
dtypes: object(11)
memory usage: 22.5+ KB


In [127]:
imdb_errors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 203 entries, 0 to 202
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tconst  203 non-null    object
dtypes: object(1)
memory usage: 3.2+ KB


In [128]:
dups_df = imdb_scrape[imdb_scrape['tconst'].isin(imdb_scrape['tconst'].value_counts()[imdb_scrape['tconst'].value_counts()>2].index)]
print(dups_df.shape)
dups_df.head(30)

(0, 11)


,tconst,newurl,rating,votes,year,mpaarating,runtime,rlsdt,budget,worldwide,soup_text


In [129]:
dups_df = imdb_errors[imdb_errors['tconst'].isin(imdb_errors['tconst'].value_counts()[imdb_errors['tconst'].value_counts()>2].index)]
print(dups_df.shape)
dups_df.head(30)

(0, 1)


,tconst


In [130]:
imdb_scrape.head()

,tconst,newurl,rating,votes,year,mpaarating,runtime,rlsdt,budget,worldwide,soup_text
0,tt0115433,https://www.imdb.com/title/tt0115433,5.7,110K,1996,G,1h 43m,"November 27, 1996 (United States)","$75,000,000 (estimated)","$320,689,294","[html, [[<meta content=""width=device-width"" na..."
1,tt0211181,https://www.imdb.com/title/tt0211181,4.8,37K,2000,G,1h 40m,"November 22, 2000 (United States)","$85,000,000 (estimated)","$183,611,771","[html, [[<meta content=""width=device-width"" na..."
2,tt0120623,https://www.imdb.com/title/tt0120623,7.2,285K,1998,G,1h 35m,"November 25, 1998 (United States)","$120,000,000 (estimated)","$363,258,859","[html, [[<meta content=""width=device-width"" na..."
3,tt1067106,https://www.imdb.com/title/tt1067106,6.8,113K,2009,PG,1h 36m,"November 6, 2009 (United States)","$200,000,000 (estimated)","$325,286,646","[html, [[<meta content=""width=device-width"" na..."
4,tt0113198,https://www.imdb.com/title/tt0113198,6.9,52K,1995,G,1h 18m,"April 7, 1995 (United States)","$18,000,000 (estimated)","$35,348,597","[html, [[<meta content=""width=device-width"" na..."


In [131]:
imdb_scrape.to_csv('../Bens_Data/imdb_scrape_1.csv')
imdb_errors.to_csv('../Bens_Data/imdb_errors_1.csv')

# THE NEXT MORNING..

In [142]:
imdb_scrape = pd.read_csv('../Bens_Data/imdb_scrape_1.csv')
imdb_errors = pd.read_csv('../Bens_Data/imdb_errors_1.csv')
imdb_errors_r2 = pd.DataFrame(columns=['tconst'])

In [143]:
imdb_scrape.drop(columns=['Unnamed: 0'], inplace=True)
imdb_scrape.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   tconst      239 non-null    object 
 1   newurl      239 non-null    object 
 2   rating      239 non-null    float64
 3   votes       239 non-null    object 
 4   year        239 non-null    int64  
 5   mpaarating  239 non-null    object 
 6   runtime     239 non-null    object 
 7   rlsdt       239 non-null    object 
 8   budget      239 non-null    object 
 9   worldwide   239 non-null    object 
 10  soup_text   239 non-null    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 20.7+ KB


In [144]:
imdb_errors.drop(columns=['Unnamed: 0'], inplace=True)
imdb_errors.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 203 entries, 0 to 202
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tconst  203 non-null    object
dtypes: object(1)
memory usage: 1.7+ KB


In [145]:
# Target web page:
url = "https://www.imdb.com/title/"
# Establishing the connection to the web page:
response = requests.get(url)
# response.text[:50]
soup = BeautifulSoup(response.text, 'lxml')

In [146]:
counter = 1
for const in imdb_errors['tconst']:
    
    # found1 = imdb_scrape[imdb_scrape['tconst'].str.contains(const)]
    # found2 = imdb_errors[imdb_errors['tconst'].str.contains(const)]
    # summ = found1.count().sum() + found2.count().sum()
    
    # if summ < 1:
    
        # const = "tt0115433"
        # const = "tt0029583"
        # const = "tt1067106"
        budget = ''
        worldwide = ''
        t= random.uniform(1, 5)
        time.sleep(t)
        newurl = url + const
        print(newurl)
        response = requests.get(newurl)
        soup = BeautifulSoup(response.text, 'lxml')
        print(counter)
        counter += 1
        rtspan = soup.find_all('span', {'class' : 'sc-7ab21ed2-1 jGRxWM'})
        rating = rtspan[0].get_text()

        vtdiv = soup.find_all('div', {'class' : 'sc-7ab21ed2-3 dPVcnq'})
        votes = vtdiv[0].text

        # mpaaspan = soup.find_all('span', {'class' : 'ipc-metadata-list-item__list-content-item'})
        # mpaarating = mpaaspan[2].get_text()
        try:

            metaul = soup.find_all('ul', {'data-testid' : 'hero-title-block__metadata'})
            year = metaul[0].find_all('li')[0].find_all('span')[0].get_text()
            mpaarating = metaul[0].find_all('li')[1].find_all('span')[0].get_text()
            runtime = metaul[0].find_all('li')[2].get_text()
            rlsdt = soup.find_all('li', {'data-testid' : 'title-details-releasedate'})[0].find_all('a')[1].get_text()
            try:
                budget = soup.find_all('li', {'data-testid' : 'title-boxoffice-budget'})[0].find_all('li')[0].get_text()
            except:
                pass
            try:    
                worldwide = soup.find_all('li', {'data-testid' : 'title-boxoffice-cumulativeworldwidegross'})[0].find_all('li')[0].get_text()
            except:
                pass   
            print(newurl)
            print(rating)
            print(votes) 
            print(year) 
            print(mpaarating) 
            print(runtime) 
            print(rlsdt) 
            print(budget) 
            print(worldwide) 
            imdb_scrape.loc[len(imdb_scrape.index)] = [const, newurl, rating, votes, year ,mpaarating ,runtime ,rlsdt ,budget ,worldwide, soup]

        except:
            imdb_errors_r2.loc[len(imdb_errors_r2.index)] = [const]
            print(f"ERROR ON: {const}")

        

https://www.imdb.com/title/tt0046672
1
https://www.imdb.com/title/tt0046672
7.2
33K
1954
G
2h 7m
July 20, 1955 (United States)
$9,000,000 (estimated)

https://www.imdb.com/title/tt0106868
2
https://www.imdb.com/title/tt0106868
6.5
3.6K
1993
PG
1h 48m
March 12, 1993 (United States)

$12,890,752
https://www.imdb.com/title/tt0058657
3
https://www.imdb.com/title/tt0058657
6.3
351
1964
Approved
1h 31m
March 12, 1964 (United States)


https://www.imdb.com/title/tt0043274
4
https://www.imdb.com/title/tt0043274
7.4
138K
1951
G
1h 15m
July 28, 1951 (United States)
$3,000,000 (estimated)

https://www.imdb.com/title/tt0417415
5
https://www.imdb.com/title/tt0417415
6.4
3K
2005
G
1h 40m
August 25, 2005 (Germany)

$12,775,590
https://www.imdb.com/title/tt0055740
6
https://www.imdb.com/title/tt0055740
6.6
550
1962
Not Rated
1h 33m
December 21, 1962 (United States)


https://www.imdb.com/title/tt0381006
7
https://www.imdb.com/title/tt0381006
5.0
680
2004
PG
1h 24m
July 2, 2004 (United States)

$314,40

In [147]:
imdb_scrape.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 438 entries, 0 to 437
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tconst      438 non-null    object
 1   newurl      438 non-null    object
 2   rating      438 non-null    object
 3   votes       438 non-null    object
 4   year        438 non-null    object
 5   mpaarating  438 non-null    object
 6   runtime     438 non-null    object
 7   rlsdt       438 non-null    object
 8   budget      438 non-null    object
 9   worldwide   438 non-null    object
 10  soup_text   438 non-null    object
dtypes: object(11)
memory usage: 41.1+ KB


In [148]:
imdb_errors_r2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 0 to 3
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tconst  4 non-null      object
dtypes: object(1)
memory usage: 64.0+ bytes


In [149]:
dups_df = imdb_scrape[imdb_scrape['tconst'].isin(imdb_scrape['tconst'].value_counts()[imdb_scrape['tconst'].value_counts()>2].index)]
print(dups_df.shape)
dups_df.head(30)

(0, 11)


,tconst,newurl,rating,votes,year,mpaarating,runtime,rlsdt,budget,worldwide,soup_text


In [150]:
imdb_scrape.to_csv('../Bens_Data/imdb_scrape_rnd2.csv')
imdb_errors_r2.to_csv('../Bens_Data/imdb_errors_rnd2.csv')

# ROUND 3 - Hand Checking Errors

In [7]:
imdb_scrape = pd.read_csv('../Bens_Data/imdb_scrape_rnd2.csv')
imdb_errors_r2 = pd.read_csv('../Bens_Data/imdb_errors_rnd2.csv')


In [8]:
imdb_errors_r2.head()

,Unnamed: 0,tconst
0,0,tt0057312
1,1,tt2064925
2,2,tt0058099
3,3,tt1686635


In [9]:
imdb_scrape.head()

,Unnamed: 0,tconst,newurl,rating,votes,year,mpaarating,runtime,rlsdt,budget,worldwide,soup_text
0,0,tt0115433,https://www.imdb.com/title/tt0115433,5.7,110K,1996,G,1h 43m,"November 27, 1996 (United States)","$75,000,000 (estimated)","$320,689,294","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
1,1,tt0211181,https://www.imdb.com/title/tt0211181,4.8,37K,2000,G,1h 40m,"November 22, 2000 (United States)","$85,000,000 (estimated)","$183,611,771","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
2,2,tt0120623,https://www.imdb.com/title/tt0120623,7.2,285K,1998,G,1h 35m,"November 25, 1998 (United States)","$120,000,000 (estimated)","$363,258,859","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
3,3,tt1067106,https://www.imdb.com/title/tt1067106,6.8,113K,2009,PG,1h 36m,"November 6, 2009 (United States)","$200,000,000 (estimated)","$325,286,646","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
4,4,tt0113198,https://www.imdb.com/title/tt0113198,6.9,52K,1995,G,1h 18m,"April 7, 1995 (United States)","$18,000,000 (estimated)","$35,348,597","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."


In [10]:
for const in imdb_errors_r2['tconst']:
    
    # found1 = imdb_scrape[imdb_scrape['tconst'].str.contains(const)]
    # found2 = imdb_errors[imdb_errors['tconst'].str.contains(const)]
    # summ = found1.count().sum() + found2.count().sum()
    
    skip = ['tt0035462','tt7728220','tt1686635','tt1935157','tt0344910','tt2905856','tt14608798','tt0780134','tt0087993','tt0057312','tt1130090','tt9047432','tt0066250','tt2064925','tt0096866','tt0058099','tt0246786','tt0385277']
    
    if (const in skip) == False:
    
        # const = "tt0115433"
        # const = "tt0029583"
        # const = "tt1067106"
        budget = ''
        worldwide = ''
        t= random.uniform(1, 5)
        time.sleep(t)
        newurl = url + const
        response = requests.get(newurl)
        soup = BeautifulSoup(response.text, 'lxml')
        print(newurl)
        rtspan = soup.find_all('span', {'class' : 'sc-7ab21ed2-1 jGRxWM'})
        rating = rtspan[0].get_text()

        vtdiv = soup.find_all('div', {'class' : 'sc-7ab21ed2-3 dPVcnq'})
        votes = vtdiv[0].text

        # mpaaspan = soup.find_all('span', {'class' : 'ipc-metadata-list-item__list-content-item'})
        # mpaarating = mpaaspan[2].get_text()
        # try:

        metaul = soup.find_all('ul', {'data-testid' : 'hero-title-block__metadata'})
        year = metaul[0].find_all('li')[0].find_all('span')[0].get_text()
        mpaarating = metaul[0].find_all('li')[1].find_all('span')[0].get_text()
        runtime = metaul[0].find_all('li')[2].get_text()
        rlsdt = soup.find_all('li', {'data-testid' : 'title-details-releasedate'})[0].find_all('a')[1].get_text()

        budget = soup.find_all('li', {'data-testid' : 'title-boxoffice-budget'})[0].find_all('li')[0].get_text()


        worldwide = soup.find_all('li', {'data-testid' : 'title-boxoffice-cumulativeworldwidegross'})[0].find_all('li')[0].get_text()

        print(newurl)
        print(rating)
        print(votes) 
        print(year) 
        print(mpaarating) 
        print(runtime) 
        print(rlsdt) 
        print(budget) 
        print(worldwide) 
        imdb_scrape.loc[len(imdb_scrape.index)] = [const, newurl, rating, votes, year ,mpaarating ,runtime ,rlsdt ,budget ,worldwide, soup]

        # except:
        #     imdb_errors_r2.loc[len(imdb_errors_r2.index)] = [const]
        #     print(f"ERROR ON: {const}")



In [ ]:
https://www.imdb.com/title/tt0057312 
        # Miracle of the White Stallions
        # Initial release: March 29, 1963
        # Director: Arthur Hiller
        # Box office $2,550,000 (US/ Canada)[1]
        # G  
        # Family, Live Action, Drama, Classics 
tt2064925 not a Disney Movie, should have been https://www.imdb.com/title/tt0049733/ 
tt0058099 not a Disney Movie, should have been https://www.imdb.com/title/tt2025690/
tt1686635 not a Disney Movie, should have been https://www.imdb.com/title/tt0046387/ 

In [11]:
imdb_scrape.drop(columns=['Unnamed: 0'], inplace=True)

In [12]:
imdb_scrape.head()

,tconst,newurl,rating,votes,year,mpaarating,runtime,rlsdt,budget,worldwide,soup_text
0,tt0115433,https://www.imdb.com/title/tt0115433,5.7,110K,1996,G,1h 43m,"November 27, 1996 (United States)","$75,000,000 (estimated)","$320,689,294","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
1,tt0211181,https://www.imdb.com/title/tt0211181,4.8,37K,2000,G,1h 40m,"November 22, 2000 (United States)","$85,000,000 (estimated)","$183,611,771","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
2,tt0120623,https://www.imdb.com/title/tt0120623,7.2,285K,1998,G,1h 35m,"November 25, 1998 (United States)","$120,000,000 (estimated)","$363,258,859","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
3,tt1067106,https://www.imdb.com/title/tt1067106,6.8,113K,2009,PG,1h 36m,"November 6, 2009 (United States)","$200,000,000 (estimated)","$325,286,646","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
4,tt0113198,https://www.imdb.com/title/tt0113198,6.9,52K,1995,G,1h 18m,"April 7, 1995 (United States)","$18,000,000 (estimated)","$35,348,597","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."


In [13]:
imdb_scrape[imdb_scrape['tconst']=='tt0049733']

,tconst,newurl,rating,votes,year,mpaarating,runtime,rlsdt,budget,worldwide,soup_text
337,tt0049733,https://www.imdb.com/title/tt0049733,7.4,272,1956,Passed,1h 10m,"November 6, 1956 (United States)",NaN,NaN,"<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."


In [14]:
imdb_scrape[imdb_scrape['tconst']=='tt2025690']

,tconst,newurl,rating,votes,year,mpaarating,runtime,rlsdt,budget,worldwide,soup_text
179,tt2025690,https://www.imdb.com/title/tt2025690,6.7,66K,2016,PG-13,1h 57m,"January 29, 2016 (United States)","$80,000,000 (estimated)","$52,099,090","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."


In [15]:
imdb_scrape[imdb_scrape['tconst']=='tt0046387']

,tconst,newurl,rating,votes,year,mpaarating,runtime,rlsdt,budget,worldwide,soup_text
403,tt0046387,https://www.imdb.com/title/tt0046387,6.3,871,1953,Approved,1h 32m,"July 23, 1953 (United States)",NaN,"$2,500,000","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."


In [16]:
imdb_scrape = imdb_scrape.drop(imdb_scrape[(imdb_scrape['tconst']=='tt2064925')].index)
imdb_scrape = imdb_scrape.drop(imdb_scrape[(imdb_scrape['tconst']=='tt0058099')].index)
imdb_scrape = imdb_scrape.drop(imdb_scrape[(imdb_scrape['tconst']=='tt1686635')].index)

Disney_IMDB = Disney_IMDB.drop(Disney_IMDB[(Disney_IMDB['tconst']=='tt2064925')].index)
Disney_IMDB = Disney_IMDB.drop(Disney_IMDB[(Disney_IMDB['tconst']=='tt0058099')].index)
Disney_IMDB = Disney_IMDB.drop(Disney_IMDB[(Disney_IMDB['tconst']=='tt1686635')].index)

In [17]:
# https://www.imdb.com/title/tt0057312 
#         # Miracle of the White Stallions
#         # Initial release: March 29, 1963
#         # Director: Arthur Hiller
#         # Box office $2,550,000 (US/ Canada)[1]
#         # G  
#         # Family, Live Action, Drama, Classics 

# const = 'tt0057312'
# newurl ='https://www.imdb.com/title/tt0057312'
# rating ='6.5'
# votes ='698'
# year ='1963'
# mpaarating ='G'
# runtime ='1h 58m'
# rlsdt ='March 29, 1963 (United States)'
# budget =''
# worldwide ='$2,550,000'
# soup = ''


# imdb_scrape.loc[IMDB_Scrape['tconst']=='tt0057312'] = [const, newurl, rating, votes, year ,mpaarating ,runtime ,rlsdt ,budget ,worldwide, soup]

In [18]:
# Target web page:
url = "https://www.imdb.com/title/"

const = "tt0057312"
budget = ''
mpaarating ='G'
runtime ='1h 58m'
worldwide ='$2,550,000'

newurl = url + const
response = requests.get(newurl)
soup = BeautifulSoup(response.text, 'lxml')
print(newurl)
rtspan = soup.find_all('span', {'class' : 'sc-7ab21ed2-1 jGRxWM'})
rating = rtspan[0].get_text()

vtdiv = soup.find_all('div', {'class' : 'sc-7ab21ed2-3 dPVcnq'})
votes = vtdiv[0].text

        # mpaaspan = soup.find_all('span', {'class' : 'ipc-metadata-list-item__list-content-item'})
        # mpaarating = mpaaspan[2].get_text()
        # try:

metaul = soup.find_all('ul', {'data-testid' : 'hero-title-block__metadata'})
year = metaul[0].find_all('li')[0].find_all('span')[0].get_text()
# mpaarating = metaul[0].find_all('li')[1].find_all('span')[0].get_text()
# runtime = metaul[0].find_all('li')[2].get_text()
rlsdt = soup.find_all('li', {'data-testid' : 'title-details-releasedate'})[0].find_all('a')[1].get_text()

# budget = soup.find_all('li', {'data-testid' : 'title-boxoffice-budget'})[0].find_all('li')[0].get_text()


# worldwide = soup.find_all('li', {'data-testid' : 'title-boxoffice-cumulativeworldwidegross'})[0].find_all('li')[0].get_text()

print(newurl)
print(rating)
print(votes) 
print(year) 
print(mpaarating) 
print(runtime) 
print(rlsdt) 
print(budget) 
print(worldwide) 
imdb_scrape.loc[len(imdb_scrape.index)] = [const, newurl, rating, votes, year ,mpaarating ,runtime ,rlsdt ,budget ,worldwide, soup]


https://www.imdb.com/title/tt0057312
https://www.imdb.com/title/tt0057312
6.5
701
1963
G
1h 58m
March 29, 1963 (United States)

$2,550,000


In [19]:
imdb_scrape.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 439 entries, 0 to 438
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tconst      439 non-null    object
 1   newurl      439 non-null    object
 2   rating      439 non-null    object
 3   votes       439 non-null    object
 4   year        439 non-null    object
 5   mpaarating  439 non-null    object
 6   runtime     439 non-null    object
 7   rlsdt       439 non-null    object
 8   budget      261 non-null    object
 9   worldwide   308 non-null    object
 10  soup_text   439 non-null    object
dtypes: object(11)
memory usage: 41.2+ KB


In [20]:
Disney_IMDB.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 439 entries, 0 to 441
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   orig_index      439 non-null    int64  
 1   DFL_title       439 non-null    object 
 2   tconst          439 non-null    object 
 3   titleType       439 non-null    object 
 4   primaryTitle    439 non-null    object 
 5   originalTitle   439 non-null    object 
 6   startYear       439 non-null    int64  
 7   runtimeMinutes  439 non-null    int64  
 8   genres          439 non-null    object 
 9   lower_title     439 non-null    object 
 10  averageRating   439 non-null    float64
 11  numVotes        439 non-null    float64
 12  _merge          439 non-null    object 
dtypes: float64(2), int64(3), object(8)
memory usage: 48.0+ KB


In [21]:
imdb_scrape.head()

,tconst,newurl,rating,votes,year,mpaarating,runtime,rlsdt,budget,worldwide,soup_text
0,tt0115433,https://www.imdb.com/title/tt0115433,5.7,110K,1996,G,1h 43m,"November 27, 1996 (United States)","$75,000,000 (estimated)","$320,689,294","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
1,tt0211181,https://www.imdb.com/title/tt0211181,4.8,37K,2000,G,1h 40m,"November 22, 2000 (United States)","$85,000,000 (estimated)","$183,611,771","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
2,tt0120623,https://www.imdb.com/title/tt0120623,7.2,285K,1998,G,1h 35m,"November 25, 1998 (United States)","$120,000,000 (estimated)","$363,258,859","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
3,tt1067106,https://www.imdb.com/title/tt1067106,6.8,113K,2009,PG,1h 36m,"November 6, 2009 (United States)","$200,000,000 (estimated)","$325,286,646","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
4,tt0113198,https://www.imdb.com/title/tt0113198,6.9,52K,1995,G,1h 18m,"April 7, 1995 (United States)","$18,000,000 (estimated)","$35,348,597","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."


In [22]:
Disney_IMDB.head()

,orig_index,DFL_title,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,lower_title,averageRating,numVotes,_merge
0,0,101 Dalmatians,tt0115433,movie,101 Dalmatians,101 Dalmatians,1996,103,"Adventure,Comedy,Crime",101 dalmatians,5.7,109712.0,both
1,5,102 Dalmatians,tt0211181,movie,102 Dalmatians,102 Dalmatians,2000,100,"Adventure,Comedy,Family",102 dalmatians,4.8,37056.0,both
2,10,"20,000 Leagues Under the Sea",tt0046672,movie,"20,000 Leagues Under the Sea","20,000 Leagues Under the Sea",1954,127,"Adventure,Drama,Family",20000 leagues under the sea,7.2,33109.0,both
3,24,A Bug's Life,tt0120623,movie,A Bug's Life,A Bug's Life,1998,95,"Adventure,Animation,Comedy",a bugs life,7.2,284538.0,both
4,74,A Christmas Carol,tt1067106,movie,A Christmas Carol,A Christmas Carol,2009,96,"Adventure,Animation,Comedy",a christmas carol,6.8,112582.0,both


In [23]:
imdb_scrape.to_csv('../Bens_Data/imdb_scrape_complete.csv')

In [24]:
imdb_scrape[imdb_scrape['tconst']=='tt0054195']

,tconst,newurl,rating,votes,year,mpaarating,runtime,rlsdt,budget,worldwide,soup_text
325,tt0054195,https://www.imdb.com/title/tt0054195,7.4,10K,1960,Approved,2h 14m,"May 19, 1960 (United States)",NaN,NaN,"<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."


In [25]:
imdb_scrape[imdb_scrape['tconst']=='tt0099816']

,tconst,newurl,rating,votes,year,mpaarating,runtime,rlsdt,budget,worldwide,soup_text
154,tt0099816,https://www.imdb.com/title/tt0099816,6.6,2.8K,1990,PG,1h 32m,"March 1, 1991 (United States)","NOK 60,000,000 (estimated)","$15,104,156","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."


In [26]:
Disney_IMDB[Disney_IMDB['tconst']=='tt0054195']

,orig_index,DFL_title,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,lower_title,averageRating,numVotes,_merge
224,3788,Pollyanna,tt0054195,movie,Pollyanna,Pollyanna,1960,134,"Comedy,Drama,Family",pollyanna,7.4,10132.0,both


In [27]:
Disney_IMDB[Disney_IMDB['tconst']=='tt0099816']

,orig_index,DFL_title,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,lower_title,averageRating,numVotes,_merge
255,4184,Shipwrecked,tt0099816,movie,Shipwrecked,Haakon Haakonsen,1990,92,"Adventure,Family",shipwrecked,6.6,2802.0,both


In [28]:
Disney_IMDB.drop(columns=['_merge'], inplace=True)

In [29]:
FL_Disney_IMDB_scrape = pd.merge(Disney_IMDB ,                 # left df
                          imdb_scrape,                  # right df
                          how="left",                 # left join
                          left_on='tconst',            # left column
                          right_on='tconst',    # right column
                          indicator = True,           # indicates source of each row
                          #validate = "one_to_many"    # alerts us of the relationship from left to right, incase there are dups
        )

In [30]:
FL_Disney_IMDB_scrape.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 439 entries, 0 to 438
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   orig_index      439 non-null    int64   
 1   DFL_title       439 non-null    object  
 2   tconst          439 non-null    object  
 3   titleType       439 non-null    object  
 4   primaryTitle    439 non-null    object  
 5   originalTitle   439 non-null    object  
 6   startYear       439 non-null    int64   
 7   runtimeMinutes  439 non-null    int64   
 8   genres          439 non-null    object  
 9   lower_title     439 non-null    object  
 10  averageRating   439 non-null    float64 
 11  numVotes        439 non-null    float64 
 12  newurl          439 non-null    object  
 13  rating          439 non-null    object  
 14  votes           439 non-null    object  
 15  year            439 non-null    object  
 16  mpaarating      439 non-null    object  
 17  runtime         

In [31]:
FL_Disney_IMDB_scrape['_merge'].value_counts()

both          439
left_only       0
right_only      0
Name: _merge, dtype: int64

In [32]:
dups_df = FL_Disney_IMDB_scrape[FL_Disney_IMDB_scrape['tconst'].isin(FL_Disney_IMDB_scrape['tconst'].value_counts()[FL_Disney_IMDB_scrape['tconst'].value_counts()>2].index)]
print(dups_df.shape)
dups_df.head(30)

(0, 23)


,orig_index,DFL_title,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,lower_title,...,rating,votes,year,mpaarating,runtime,rlsdt,budget,worldwide,soup_text,_merge


In [35]:
imdb_disney_soup = FL_Disney_IMDB_scrape[['tconst', 'DFL_title','newurl','soup_text']]

In [37]:
imdb_disney_soup.head()

,tconst,DFL_title,newurl,soup_text
0,tt0115433,101 Dalmatians,https://www.imdb.com/title/tt0115433,"<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
1,tt0211181,102 Dalmatians,https://www.imdb.com/title/tt0211181,"<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
2,tt0046672,"20,000 Leagues Under the Sea",https://www.imdb.com/title/tt0046672,"<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
3,tt0120623,A Bug's Life,https://www.imdb.com/title/tt0120623,"<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."
4,tt1067106,A Christmas Carol,https://www.imdb.com/title/tt1067106,"<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""..."


In [38]:
imdb_disney_soup.to_csv('../Bens_Data/imdb_disney_soup.csv')

In [40]:
FL_Disney_IMDB_scrape.head(3)

,orig_index,DFL_title,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,lower_title,...,rating,votes,year,mpaarating,runtime,rlsdt,budget,worldwide,soup_text,_merge
0,0,101 Dalmatians,tt0115433,movie,101 Dalmatians,101 Dalmatians,1996,103,"Adventure,Comedy,Crime",101 dalmatians,...,5.7,110K,1996,G,1h 43m,"November 27, 1996 (United States)","$75,000,000 (estimated)","$320,689,294","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""...",both
1,5,102 Dalmatians,tt0211181,movie,102 Dalmatians,102 Dalmatians,2000,100,"Adventure,Comedy,Family",102 dalmatians,...,4.8,37K,2000,G,1h 40m,"November 22, 2000 (United States)","$85,000,000 (estimated)","$183,611,771","<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""...",both
2,10,"20,000 Leagues Under the Sea",tt0046672,movie,"20,000 Leagues Under the Sea","20,000 Leagues Under the Sea",1954,127,"Adventure,Drama,Family",20000 leagues under the sea,...,7.2,33K,1954,G,2h 7m,"July 20, 1955 (United States)","$9,000,000 (estimated)",NaN,"<!DOCTYPE html>\n<html lang=""en-US"" xmlns:fb=""...",both


In [41]:
FL_Disney_IMDB_scrape.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 439 entries, 0 to 438
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   orig_index      439 non-null    int64   
 1   DFL_title       439 non-null    object  
 2   tconst          439 non-null    object  
 3   titleType       439 non-null    object  
 4   primaryTitle    439 non-null    object  
 5   originalTitle   439 non-null    object  
 6   startYear       439 non-null    int64   
 7   runtimeMinutes  439 non-null    int64   
 8   genres          439 non-null    object  
 9   lower_title     439 non-null    object  
 10  averageRating   439 non-null    float64 
 11  numVotes        439 non-null    float64 
 12  newurl          439 non-null    object  
 13  rating          439 non-null    object  
 14  votes           439 non-null    object  
 15  year            439 non-null    object  
 16  mpaarating      439 non-null    object  
 17  runtime         

In [43]:
FL_Disney_IMDB_scrape.drop(columns=['orig_index','soup_text','_merge'], inplace=True)

In [44]:
FL_Disney_IMDB_scrape.to_csv('../Bens_Data/Disney_imdb_scrape_final.csv')